In [3]:
def plotMRMS(filename = None, z = None, output = None, directory = "./", show = True, debug=False, shapefiles=None, thin=3):
    from netcdftime import utime
    from mpl_toolkits.axes_grid.inset_locator import inset_axes
    from mpl_toolkits.basemap import Basemap
    import ctables
    
    sec_utime = utime("seconds since 1970-01-01 00:00:00")
    
    f = ncdf.Dataset(filename, "r")
    dbz = f.variables['mrefl_mosaic'][0,:,::thin,::thin]
    dbz = N.ma.masked_array(dbz, mask=(dbz == f.MissingData))
    
    hgt = f.variables['Ht'][:]
    lat = f.variables['Lat'][::thin]   
    lon = f.variables['Lon'][::thin]
    timestamp = f.variables['time'][0] 
    
    if debug:
        print lat[0],lat[-1],lat.shape
        print lon[0],lon[-1],lon.shape
        print dbz.shape

    if z <= 0:
        k = -z
    else:
        kz = hgt[hgt >= z]
        if kz.size > 0:  
            k = kz[0] 
        else:
            k = 0
            
    dbzslice = dbz[k]

    lons, lats = N.meshgrid(lon, lat[::-1])
    
    fig = P.figure(figsize=(12,12))
    ax = fig.add_subplot(111)

    map = Basemap(projection='lcc', llcrnrlon=-99.75,llcrnrlat=33.75,urcrnrlon=-96.,urcrnrlat=36.75, lat_0=34.,lon_0=-98., area_thresh=1,suppress_ticks=True)
    x, y = map(lons,lats)
    clevels = N.arange(0,75,5)
    plot = map.contourf(x, y, dbzslice, clevels, cmap=ctables.NWSRef)
    cbar = map.colorbar(plot, location='bottom', pad="5%")
    cbar.set_label('dBZ')

    ax.set_title("Time:  %s       dBZ at %4.2f km MSL" % (sec_utime.num2date(timestamp), 0.001*hgt[k]), size='xx-large')
    ax.set_aspect('equal')
    ax.set_xlabel('Lon (km)')
    ax.set_ylabel('Lat (km)')
    
    try:
        if shapefiles:
            shapelist = shapefiles
        else:
            shapelist = os.getenv("PYESVIEWER_SHAPEFILES").split(":")

        if len(shapelist) > 0:

            for item in shapelist:
                items = item.split(",")
                shapefile  = items[0]
                color      = items[1]
                line_width = items[2]
                print shapefile, color, line_width

                map.readshapefile(shapefile,'GIS_INFO',drawbounds=True,linewidth=line_width,color=color)

    except OSError:
        print "GIS_PLOT:  NO SHAPEFILE ENV VARIABLE FOUND "
        pass

    output_filename = "%s_%4.2f.png" % (os.path.join(directory,"MRMS_"+str.replace(sec_utime.num2date(timestamp).isoformat(),"T","_")),hgt[k])
    if output != None:
        print "\n Saving file %s" % (output_filename)
        P.savefig(output_filename, format="png", dpi=300)
    if show == True:
        P.show()
        
    P.close()

        

In [ ]:
import glob
dir = "/Volumes/Master/Users/wicker/ownCloud/RadarData/24May2011/MRMS_24May2011_1930_2130_cdf"
shapefiles = "/Volumes/Master/Users/wicker/Library/ShapeFiles/tornado_tracks/24May_2011/El_Reno_B1_B2,black,1.0:/Volumes/Master/Users/wicker/Library/ShapeFiles/shapefiles/OKCNTYD/OKCNTYD,gray,2.0"
files = glob.glob(os.path.join(dir,"*.netcdf"))
for file in files:
    plotMRMS(filename=os.path.join(dir,file),z=-5, directory='/Volumes/Master/Users/wicker/test2',show=False,output=True)
    print("File %s completed" % file)


/Volumes/Master/Users/wicker/Library/ShapeFiles/tornado_tracks/24May_2011/El_Reno_B1_B2 black 1.0
/Volumes/Master/Users/wicker/Library/ShapeFiles/shapefiles/OKCNTYD/OKCNTYD gray 2.0

 Saving file /Volumes/Master/Users/wicker/test2/MRMS_2011-05-24_19:30:00_1750.00.png
File /Volumes/Master/Users/wicker/ownCloud/RadarData/24May2011/MRMS_24May2011_1930_2130_cdf/20110524-193000.netcdf completed

In [4]:
import glob
file = "/Volumes/Master/Users/wicker/ownCloud/RadarData/24May2011/MRMS_24May2011_1930_2130_cdf/20110524-200000.netcdf"
shapefiles = "/Volumes/Master/Users/wicker/Library/ShapeFiles/tornado_tracks/24May_2011/El_Reno_B1_B2,black,1.0:/Volumes/Master/Users/wicker/Library/ShapeFiles/shapefiles/OKCNTYD/OKCNTYD,gray,2.0"
plotMRMS(filename=file,z=-6, directory='/Volumes/Master/Users/wicker/test2',show=True,output=False)


/Volumes/Master/Users/wicker/Library/ShapeFiles/tornado_tracks/24May_2011/El_Reno_B1_B2 black 1.0
/Volumes/Master/Users/wicker/Library/ShapeFiles/shapefiles/OKCNTYD/OKCNTYD gray 2.0

 Saving file /Volumes/Master/Users/wicker/test2/MRMS_2011-05-24_20:00:00_2000.00.png


In [9]:
import glob
dir = "/Volumes/Master/Users/wicker/ownCloud/RadarData/24May2011/MRMS_24May2011_1930_2130_cdf"
files = glob.glob(os.path.join(dir,"*.netcdf"))
for file in files:
    cmd = "ncpdq -a time %s %s/tmp/%s" % (file, dir, os.path.split(file)[-1])
    os.system(cmd)
    
